# Importing dataset

In [1]:
import pandas as pd 
import numpy as np

df = pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [2]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

## Using normal process

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']), df['has_covid'], test_size=0.2, random_state=42)

In [4]:
X_train

,age,gender,fever,cough,city
55,81,Female,101.0,Mild,Mumbai
88,5,Female,100.0,Mild,Kolkata
26,19,Female,100.0,Mild,Kolkata
42,27,Male,100.0,Mild,Delhi
69,73,Female,103.0,Mild,Delhi
...,...,...,...,...,...
60,24,Female,102.0,Strong,Bangalore
71,75,Female,104.0,Strong,Delhi
14,51,Male,104.0,Mild,Bangalore
92,82,Female,102.0,Strong,Kolkata


#### Filling missing values in Fever column

In [5]:
from sklearn.impute import SimpleImputer
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.transform(X_test[['fever']])

Ordinal encoding for cough column


In [6]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(categories = [['Mild', 'Strong']])

X_train_cough = oe.fit_transform(X_train[['cough']])

X_test_cough = oe.transform(X_test[['cough']])

### Applying OneHotEncoding to column city and gender

In [7]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [8]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first', sparse_output=False)

In [9]:
X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']])

In [10]:
X_test_gender_city = ohe.transform(X_test[['gender', 'city']])

In [11]:
X_train_gender_city

array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 1., 0.],
       [1., 0., 1., 0.],
       [0., 0., 0., 0.],
       [1., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 1.],
       [1., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 0., 0., 1.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 1.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],


In [12]:
## Extracting age
X_train_age = X_train[['age']].values
X_test_age = X_test[['age']].values

In [13]:
X_test_age

array([[17],
       [83],
       [68],
       [72],
       [20],
       [50],
       [71],
       [14],
       [75],
       [60],
       [64],
       [15],
       [34],
       [26],
       [59],
       [65],
       [80],
       [ 8],
       [25],
       [83]])

#### Now combining together

In [14]:
X_train_transformed = np.concatenate((X_train_age, X_train_gender_city, X_train_fever, X_train_cough), axis=1)
X_test_transformed = np.concatenate((X_test_age, X_test_gender_city, X_test_fever, X_test_cough), axis=1)

In [15]:
X_train_transformed.shape

(80, 7)

# Using Column transformer

In [16]:
from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer(transformers=[
    ('tf1', SimpleImputer(), ['fever']),
    ('tf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tf3', OneHotEncoder(sparse_output=False, drop='first'), ['gender', 'city'])
], remainder='passthrough')

In [18]:
X_train_ct = transformer.fit_transform(X_train)
X_test_ct = transformer.transform(X_test)

In [19]:
X_train_ct.shape, X_test_ct.shape

((80, 7), (20, 7))